In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

# 문제 S-1: RDD 데이터를 LabeledPoint로 변환하기

### 문제
머신러닝은 사람이 경험을 통해 배우는 것과 비슷하게 **과거 데이터로부터 학습**을 한다. 학습이란 어렵게 생각할 필요 없이, 과거 데이터에서 수학적이나 알고리즘을 활용하여 어떤 패턴을 찾아내는 것이다. spark에서 제공한 **데이터 파일 data/mllib/sample_svm_data.txt을 읽어서 훈련데이터**를 만들어 보자.

데이터를 읽어 보면, 맨 처음 값은 label에 해당하고, 다음은 일련의 수로 구성된다. 이로부터 RDD를 생성하고, label, features를 구성하여 Labeled Point로 만든다.

> 1 0 2.52078447201548 0 0 0 2.004684436494304 2.000347299268466 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0  
...

### Python으로 파일 읽기
Spark를 다운로드하고 압축을 풀어 설치한 경우, 환경변수 SPARK_HOME의 경로를 설정해 주고 해당 파일을 읽도록 한다. 파일을 읽을 때는, 가급적 이와 같이 설정경로를 사용해 입력오류를 줄이도록 하는 것이 좋다. 다운로드 받지 않았다면, 아파치 github을 방문해서, https://github.com/apache/spark/ 아래 data/mllib 폴더로 가보면 해당 파일을 찾을 수 있다.

In [3]:
import os
_fsvm=os.path.join(os.environ["SPARK_HOME"],'data','mllib','sample_svm_data.txt')

In [5]:
try:
    _f=open(_fsvm,'r')
    _lines=_f.readlines()
    _f.close()
except:
    print("An exception occurred")

In [6]:
_lines[0]

'1 0 2.52078447201548 0 0 0 2.004684436494304 2.000347299268466 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0\n'

### Spark에서 RDD 생성

In [7]:
_rdd=spark.sparkContext.textFile(_fsvm)\
    .map(lambda line: [float(x) for x in line.split()])

In [10]:
_rdd.take(2)[0] #맨 앞에만 lable, 나머지는 features

[1.0,
 0.0,
 2.52078447201548,
 0.0,
 0.0,
 0.0,
 2.004684436494304,
 2.000347299268466,
 0.0,
 2.228387042742021,
 2.228387042742023,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

### LabeledPoint 생성
첫 열은 lable로, 나머지는 features로 생성

In [11]:
from pyspark.mllib.regression import LabeledPoint

_trainRdd0=_rdd.map(lambda line:LabeledPoint(line[0], line[1:]))

In [12]:
_trainRdd0.take(1)

[LabeledPoint(1.0, [0.0,2.52078447201548,0.0,0.0,0.0,2.004684436494304,2.000347299268466,0.0,2.228387042742021,2.228387042742023,0.0,0.0,0.0,0.0,0.0,0.0])]

In [21]:
_trainRdd=spark.sparkContext.textFile(_fsvm)\
    .map(lambda line: [float(x) for x in line.split()])\
    .map(lambda p:LabeledPoint(p[0], p[1:]))

In [22]:
_trainRdd.take(1)

[LabeledPoint(1.0, [0.0,2.52078447201548,0.0,0.0,0.0,2.004684436494304,2.000347299268466,0.0,2.228387042742021,2.228387042742023,0.0,0.0,0.0,0.0,0.0,0.0])]

### 정리하면....

In [23]:
def createLP(line):
    p = [float(x) for x in line.split()]
    return LabeledPoint(p[0], p[1:])

_rdd=spark.sparkContext.textFile(_fsvm)
trainRdd = _rdd.map(createLP)

trainRdd.take(1)

[LabeledPoint(1.0, [0.0,2.52078447201548,0.0,0.0,0.0,2.004684436494304,2.000347299268466,0.0,2.228387042742021,2.228387042742023,0.0,0.0,0.0,0.0,0.0,0.0])]